In [19]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
from tqdm import tqdm, trange
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [20]:
from bs4 import BeautifulSoup
import re
from uuid import uuid4

In [21]:
class Crawler:

    def get_html(self):
        pass

    def get_card_info(self):
        pass

    def get_links(self, page):
        pass

In [ ]:
class CrawlerEda(Crawler):
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--headless=new")

        self.browser = webdriver.Chrome(options=chrome_options)

    def get_html(self, page):

        url = f"https://eda.ru/recepty?page={page}"
        #browser.implicitly_wait(30)
        self.browser.get(url)
        wait = WebDriverWait(self.browser, 10)
        soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        return soup

    def get_card_info(self, link):
        action = ActionChains(self.browser)
        count = 0

        #linker = linkers[number]['link']
        #browser.get(linker)
        self.browser.get(link)

        wait = WebDriverWait(self.browser, 10)
        self.browser.implicitly_wait(30)

        soup_card = BeautifulSoup(self.browser.page_source, 'html.parser')
        return soup_card

    def get_links(self,first_page, last_page):
        links = []
        for i in trange(first_page, last_page, ncols=80, desc='Total'):
            html = CrawlerEda().get_html(i)
            links.extend(ParserEda(html).get_cards())
            #print(links)
        return links

In [60]:
class CrawlerPovar(Crawler):
    def get_html(self, page):
        url = f'https://www.povarenok.ru/recipes/show/{page}/'
        '''
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--headless=new")

        self.browser = webdriver.Chrome(options=chrome_options)
        self.browser.get(url)

        wait = WebDriverWait(self.browser, 10)
        self.browser.implicitly_wait(30)

        soup_card = BeautifulSoup(self.browser.page_source, 'html.parser')
        '''
        #BeautifulSoup(requests.get(url).text, 'html.parser')
        return BeautifulSoup(requests.get(url).text, 'html.parser')

    def check_link(self, page):
        url = f'https://www.povarenok.ru/recipes/show/{page}/'
        html = self.get_html(page)
        if html.find('h1', itemprop="name") != None:
            photo = html.find('img', itemprop="image").get('src')
            page += 1
            DBHelper().insert("link", ["link", "photo"], [url, photo])
        return page

In [24]:
class Parser:

    def __init__(self, html, helper):
        helper.generic_create_table('recipe',
                    [('id', 'INTEGER')],
                    None,
                    None,
                    ['name_dish'],
                    ['count_portion', 'cook_time', 'manual'],
                    True)

        helper.generic_create_table('ingredient',
                    [('id', 'INTEGER')],
                    None,
                    None,
                    ['name_ingr'],
                    None,
                    True)

        #table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto
        helper.generic_create_table('recipe_ingredients',
                    [('id', 'INTEGER')],
                    ['recipe_id', 'ingr_id'],
                    [('recipe', 'id'), ('ingredient', 'id')],
                    None,
                    ['count'],
                    True)

        helper.generic_create_table('category',
                    [('id', 'INTEGER')],
                    None,
                    None,
                    ['name_cat'],
                    None,
                    True)

        #table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto
        helper.generic_create_table('recipe_categories',
                    [('id', 'INTEGER')],
                    ['recipe_id', 'cat_id'],
                    [('recipe', 'id'), ('category', 'id')],
                    None,
                    None,
                    True)

        helper.get_unique('recipe_categories', 'unique_index_recipe_cat', ['recipe_id', 'cat_id'])

        #self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params
        helper.create_table('calories', 'recipe_id', 'recipe', 'recipe_id', 'id', ['calories', 'proteins', 'fats', 'carbohydrates'])

        self.html = html

    def get_cards(self):
        pass

    def __get_ingredients(self, id_):
        pass

    def __get_calories(self, id_):
        pass

    def __get_manual(self):
        pass

    def __get_category(self, id_):
        pass

    def get_recipe(self, id_):
        pass

In [25]:
class ParserEda(Parser):

    def get_cards(self):
        linkers = []
        for recipe in self.html.findAll('div', class_='emotion-1f6ych6'):
            try:
                link = "https://eda.ru"+recipe.find('a', class_='emotion-12sjte8').get('href')
                photo = str(recipe.find('img', class_='emotion-1bi124x').get('src')).replace('c180x180', 'c500x500')
                #print(link)
                #print(photo)
                linkers.append(Link(link, photo))
            except AttributeError:
                print("There's no item with that code")
        #print(len(linkers))
        return linkers

    def get_recipe(self, id_):
        #print("get_recipe")

        card = self.html.find('div', class_='emotion-2k9cfu')

        name_dish = card.find('div', class_='emotion-19rdt1j').find('h1', class_='emotion-gl52ge').text
        #print(name_dish)

        count_portion = str(re.sub(r'\bпорц[ий|ия|ии]*\b', '', card.find('div', class_='emotion-1047m5l').text).strip())
        cook_time = card.find('div', class_='emotion-18q6z5o').find('div', class_='emotion-my9yfq').text
        #print(cook_time)

        manual = self.__get_manual().replace('\'', '\'\'')

        recipe = Recipe(name_dish, count_portion, cook_time, manual)

        #time.sleep(2)
        #table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto


        flag = self.helper.insert('recipe', ['id', 'name_dish', 'count_portion', 'cook_time', 'manual'],
                                [id_, recipe.name, recipe.count_portion, recipe.cook_time, recipe.manual ])

        #self, table_name, column_args, arg
        #находим айдишник нужного рецепта
        #recipe_ = helper.get('recipe', ['name_dish', 'count_portion'], [name_dish, count_portion])
        #id_ = recipe_[0][0]

        if flag:
            self.__get_categories(id_)
            self.__get_ingredients(id_)
            self.__get_calories(id_)

        #name_dish, count_portion, cook_time, ingredients, categories, manual

    #autoincrement
    def __get_ingredients(self, id_):
        all_ingredients = {}
        name_ingredients = []

        for ing in self.html.findAll('div', class_='emotion-bjn8wh'):
            name_ingredients.append(ing.text)

        for i in range (len(name_ingredients)):
            all_ingredients[name_ingredients[i]] = self.html.findAll('span', class_='emotion-bsdd3p')[i].text
            #print(all_ingredients[name_ingredients[i]], name_ingredients[i])

        self.helper.get_unique('recipe_ingredients', 'unique_index_recipe_ingr', ['recipe_id', 'ingr_id'])

        for name in name_ingredients:
            self.helper.insert('ingredient', ['name_ingr'], [str(name).replace("'", "''")])

        for i in range (len(name_ingredients)):
            #self, table_name, column_args, args
            ingr_ = self.helper.get('ingredient', ['name_ingr'], [str(name_ingredients[i]).replace("'", "''")])
            self.helper.insert('recipe_ingredients',
                          ['recipe_id', 'ingr_id', 'count'],
                          [id_, ingr_[0][0],
                           all_ingredients[ingr_[0][1]]])

        return all_ingredients

    #autoincrement
    def __get_categories(self, id_):
        categories = []

        #дурацкие категории
        bad_categories = ['Главная', 'Проверено «Едой»', 'Пошаговые рецепты', 'Видеорецепты', 'Рецепты с историей']

        for category in self.html.find('div', class_='emotion-fq1t0q').findAll('span', class_='emotion-1h6i17m'):
            if category.text not in bad_categories:
                categories.append(category.text)


        for category in categories:
            self.helper.insert('category', ['name_cat'], [category])
            self.helper.insert('recipe_categories', ['recipe_id', 'cat_id'],
                          [id_, self.helper.get('category', ['name_cat'],
                                          [category])[0][0]])

        return categories

    #
    def __get_calories(self, id_):
        calories = []

        for j in range(4):
            calories.append(self.html.findAll('div', class_='emotion-8fp9e2')[j].text)

        calories_recipe = Calories(calories[0], calories[1], calories[2], calories[3])

        self.helper.insert('calories', ['recipe_id', 'calories', 'proteins', 'fats', 'carbohydrates'], [id_,
                          calories_recipe.calories,
                          calories_recipe.proteins,
                          calories_recipe.fats,
                          calories_recipe.carbohydrates])

        return calories_recipe

    def __get_manual(self):
        manual = ''

        for point in self.html.findAll('span', class_='emotion-1dvddtv'):
            manual += point.text.replace(u'\xa0', u' ') + '\n' + '\n'

        return manual

In [53]:
class ParserPovar(Parser):

    # def get_info(self, link, j):
    #    if self.html.find('h1', itemprop="name") != None:
    #        photo = self.html.find('img', itemprop="image").get('src')
    #        j += 1
    #        DBHelper().insert("link", "id", [str(j), link, photo])
    #    return j

    def get_ingredients(self, id_):
        ingredients = []
        html = self.html.find('div', class_="ingredients-bl")
        for ing in html.findAll('li', itemprop="recipeIngredient"):
            ingredients = ing.findAll('span')
            name = ingredients[0].text
            count = ingredients[len(ingredients) - 1].text
            self.helper.insert('ingredient', ['name_ingr'], [str(name).replace("'", "''")])
            ingr_ = self.helper.get('ingredient', ['name_ingr'], [str(name).replace("'", "''")])
            self.helper.insert('recipe_ingredients',
                          ['recipe_id', 'ingr_id', 'count'],
                          [id_, ingr_[0][0],
                           count])
            #print(name, count)

        return ingredients

    def get_manual(self):
        manual = ''
        html = self.html.find('ul', itemprop="recipeInstructions")
        for rec in html.findAll('li', class_="cooking-bl"):
            manual += '' + rec.find('p').text
        #print(manual)
        if manual == '':
            k = 1 / 0
        return manual

    def get_calories(self, id_):
        html = self.html.find('div', itemprop="nutrition")
        calories_ = html.find('strong', itemprop="calories").text
        protein_ = html.find('strong', itemprop="proteinContent").text
        fat_ = html.find('strong', itemprop="fatContent").text
        carbohydrate_ = html.find('strong', itemprop="carbohydrateContent").text
        calories = " ".join(calories_.split())
        protein = " ".join(protein_.split())
        fat = " ".join(fat_.split())
        carbohydrate = " ".join(carbohydrate_.split())
        all = [calories, protein, fat, carbohydrate]
        #print(calories, protein, fat, carbohydrate)
        self.helper.insert('calories', ['recipe_id', 'calories', 'proteins', 'fats', 'carbohydrates'], [id,
                          calories,
                          protein,
                          fat,
                          carbohydrate])
        return all

    def check_ingredients(self, html):
        html.find('div', class_="ingredients-bl")

    def get_category(self, id_):
        category = []
        flag = False
        i = 0
        html = self.html.find('div', class_="article-breadcrumbs")
        for cat in html.findAll('span', itemprop="recipeCategory"):
            cate = " ".join(cat.find('a').text.split())
            category.append(cate)
            #print(category[i])
            #self.helper.insert('category', ['name_cat'], [category[i]])
            a = self.helper.get('category', ['name_cat'], [category[i]])
            if a != None and len(a) != 0:
                flag = True
                self.helper.insert('recipe_categories', ['recipe_id', 'cat_id'],
                            [id_, a[0][0]])
            i += 1
        if not flag:
            self.helper.insert('category', ['name_cat'], [category[0]])
            self.helper.insert('recipe_categories', ['recipe_id', 'cat_id'],
                            [id_, self.helper.get('category', ['name_cat'],
                                                 [category[0]])[0][0]])
        return category

    def get_recipe(self, id_, link):
        try:
            html = self.html.find('article', class_="item-bl item-about")
            self.check_ingredients(html)
            manual = self.get_manual(html, id_)
            name_dish = html.find('h1', itemprop="name").text
            cook_time = html.find('time', itemprop="totalTime").text
            count_portion = html.find('span', itemprop="recipeYield").text
            photo = html.find('img', itemprop="image").get('src')
            flag = self.helper.insert('recipe', ['id', 'name_dish', 'count_portion', 'cook_time', 'manual'],
                            [id_, name_dish, count_portion, cook_time, manual])
            if flag:
                category = self.get_category(html, id_)
                energy = self.get_calories(html, id_)
                ingredients = self.get_ingredients(html, id_)
        except:
            pass


In [27]:
class Recipe:
    def __init__(self, name_dish, count_portion, cook_time, manual):
        self.name = name_dish
        self.count_portion = count_portion
        self.cook_time = cook_time
        self.manual = manual

In [28]:
class Category:
    def __init__(self, name_cat):
        self.name = name_cat

In [29]:
class Ingredient:
    def __init__(self, name_ingr):
        self.name = name_ingr

In [30]:
class Calories: #можно взять по айдишнику рецепта
    def __init__(self, calories, proteins, fats, carbohydrates):
        self.calories = calories
        self.proteins = proteins
        self.fats = fats
        self.carbohydrates = carbohydrates

In [31]:
class Link:
    def __init__(self, link, photo):
        self.link = link
        self.photo = photo

In [32]:
from sqlite3 import IntegrityError


class DBHelper:

    def __init__(self, db_name = "recipe.db"):
        self.dbname = f"sqlite3:/{db_name}"
        self.conn = sqlite3.connect(db_name)

    #for link, recipe, ingredient and category
    def create_table_simple(self, table_name, params):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params) - 1] + " TEXT "

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY AUTOINCREMENT , {column})"
        #print(new_table)
        try:
            #print(new_table)
            self.conn.execute(new_table)
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    #for calories and recipe_ingredients
    def create_table(self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params, auto = False):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params)-1] + " TEXT "

        foreign_key = f"FOREIGN KEY ({foreign_col})  REFERENCES {foreign_table_name} ({another_foreign_col})"

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} INTEGER PRIMARY KEY {('', 'AUTOINCREMENT')[auto == True]} , {column}, {foreign_key})"

        try:
            #print(new_table)
            self.conn.execute(new_table)
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    #id autoincrement
    def generic_create_table(self, table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto):

        new_table = ""

        begin = f"CREATE TABLE IF NOT EXISTS {table_name} ("

        primary = ""
        column = " "

        if len(primary_keys) > 1:
            for i in range(len(primary_keys) - 1):
                primary += primary_keys[i][0] + ", "
                column += primary_keys[i][0] + f" {primary_keys[i][1]}, "

            primary += primary_keys[len(primary_keys) - 1][0]
            column += primary_keys[len(primary_keys) - 1][0] + f" {primary_keys[len(primary_keys) - 1][1]}, "

            new_table += f"PRIMARY KEY ({primary})"
        else:
            begin += f"{primary_keys[0][0]} {primary_keys[0][1]} PRIMARY KEY {('', 'AUTOINCREMENT')[auto]}"
            column += ", " + column

        if foreign_keys is not None and len(foreign_keys) != 0:
            reference = ""
            for i in range(len(references) - 1):
                reference += f"FOREIGN KEY ({foreign_keys[i]}) REFERENCES {references[i][0]}({references[i][1]}), "
                column += foreign_keys[i] + " TEXT, "
            reference += f"FOREIGN KEY ({foreign_keys[len(foreign_keys) - 1]}) REFERENCES" \
                         f" {references[len(references) - 1][0]}({references[len(references) - 1][1]})"
            column += foreign_keys[len(foreign_keys) - 1] + " TEXT, "

            new_table += f", {reference}"

        if unique_args is not None and len(unique_args) != 0:
            unique = ""
            for i in range(len(unique_args) - 1):
                unique += unique_args[i] + ", "
                column += unique_args[i] + " TEXT, "
            unique += unique_args[len(unique_args) - 1]
            column += unique_args[len(unique_args) - 1] + " TEXT, "

            new_table += f", UNIQUE ({unique})"

        if other_args is not None and len(other_args) != 0:
            for i in range(len(other_args) - 1):
                column += other_args[i] + " TEXT, "
            column += other_args[len(other_args)-1] + " TEXT, "

        new_table = begin + (column, column[:-2])[len(primary_keys) == 1] + new_table + ")"

        try:
            #print(new_table)
            self.conn.execute(new_table)
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def get_unique(self, table_name, name, unique_columns):
        column = ""
        for i in range(len(unique_columns) - 1):
            column += unique_columns[i] + ", "
        column += unique_columns[len(unique_columns)-1]

        try:
            #print("Unique")
            self.conn.execute(f"CREATE UNIQUE INDEX {name} ON {table_name}({column})")
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def insert_check(self, table_name, primary_key, primary_value, args):
        arguments = f"'{primary_value}', '"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        new_insert = f"INSERT INTO {table_name} SELECT * FROM (SELECT {arguments}) AS tmp " \
                     f"WHERE NOT EXISTS ( SELECT {primary_key} FROM {table_name} WHERE {primary_key} = '{primary_value}'" \
                     f") LIMIT 1"
        try:
            #print(new_insert)
            self.conn.execute(new_insert)
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def insert(self, table_name, column_args, args):
        arguments = "'"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        columns = ""
        for i in range(len(column_args) - 1):
            columns += str(column_args[i]) + ", "
        columns += str(column_args[len(column_args) - 1])

        new_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({arguments})"
        #print(new_insert)

        try:
            #print(new_insert)
            self.conn.execute(new_insert)
            #print("Выполнилось")
            self.conn.commit()
            return True
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()
            return False

    def print_info(self, table_name):
        new_get = f"SELECT * FROM {table_name}"
        cursor = self.conn.execute(new_get)

        #print("Выполнилось")

        return [row for row in cursor]

    def get(self, table_name, column_args, args):
        query = ""
        for i in range(len(column_args) - 1):
            query += column_args[i] + " = '" + args[i] + "' AND "
        query += column_args[len(column_args) - 1] + " = '" + args[len(column_args) - 1] + "'"

        new_get = f"SELECT * FROM {table_name} WHERE {query}"
        #print(new_get)

        cursor = self.conn.execute(new_get)

        return [row for row in cursor]

    def rollback(self):
        self.conn.rollback()

    #def update(self, flat_id, updates):


    #def delete(self, flat_id):


In [65]:
class Start:
    def add_links_Eda(self):
        for j in range(1):
            linkers = CrawlerEda().get_links(1, 720)
            print(len(linkers))
            for i in trange(len(linkers), ncols=80, desc='Total'):
                DBHelper().insert("link", ["link", "photo"], [linkers[i].link, linkers[i].photo])

    def add_links_Povar(self):
        j = 0
        for i in trange(50000, 152671, ncols=80, desc='Total'):
            print(i, j)
            j = CrawlerPovar().get_info(j)
            if j > 40000:
                return

    def parse_links(self):
        helper = DBHelper()
        links = helper.print_info('link')
        parsers = [ParserEda, ParserPovar]
        for i in trange(len(links), ncols=80, desc='Total'):
            l = links[i]
            #print(l)
            link = Link(l[1], l[2])
            #print(l[1], l[2])
            for parser in parsers:
                try:
                    parser(crawler.get_card_info(link.link), helper).get_recipe(l[0])
                except:
                    continue

In [64]:
Start().parse_links()

Total: 100%|██████████████████████████| 32035/32035 [00:00<00:00, 214028.85it/s]


In [34]:
linkers = CrawlerEda().get_links(1, 5)
print(len(linkers))
for i in linkers:
    print(i.id, i.link, i.photo)

TypeError: __init__() missing 1 required positional argument: 'helper'

In [ ]:
#table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto
DBHelper("recipe.db").generic_create_table('link',
                                           [('id', 'INTEGER')],
                                           None,
                                           None,
                                           ['link'],
                                           ['photo'],
                                           True)

In [ ]:
DBHelper("recipe.db").print_info("ingredient")

In [ ]:
crawler = CrawlerEda()
ParserEda(crawler.get_card_info('https://eda.ru/recepty/osnovnye-blyuda/zapechennij-kartofel-po-francuzski-la-kartofel-28620')).get_recipe(1229)

In [ ]:
crawler = CrawlerEda()
ParserEda(crawler.get_card_info('https://eda.ru/recepty/vypechka-deserty/kievskiy-tort-173835')).get_recipe(2339)

In [ ]:
crawler = CrawlerEda()
ParserEda(crawler.get_card_info('https://eda.ru/recepty/osnovnye-blyuda/pasta-s-krabom-138443')).get_recipe(4507)

In [ ]:
pov_helper = DBHelper("new.db")
pov_links = pov_helper.print_info("link")
print(pov_links)

In [ ]:
#Добавление базы данных с линками из поваренка к базе с едой
eda_helper = DBHelper()
pov_helper = DBHelper("new.db")
pov_links = pov_helper.print_info("link")
for i in trange(len(pov_links), ncols=80, desc='Total'):
    l = pov_links[i]
    #print(l)
    link = Link(l[1], l[2])
    #print(l[0], l[1], l[2])
    eda_helper.insert('link', ['link', 'photo'], [link.link, link.photo])
    #ParserEda(crawler.get_card_info(link.link)).get_recipe(l[0])